In [7]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(2)

In [37]:
df.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,957.9,957.9,957.1,0.0,21.7,21.7,23.7,21.7,23.7,...,100.0,130.5,3.75,2.10,TUPA,-21.927251,-50.490251,498.0,2021-01-01 00:00:00+00:00,nao chove
1,0.0,958.0,958.3,957.9,0.0,21.4,21.4,21.7,21.4,21.6,...,100.0,109.0,5.40,3.35,TUPA,-21.927251,-50.490251,498.0,2021-01-01 01:00:00+00:00,nao chove
2,0.0,957.4,958.0,957.4,0.0,21.2,21.2,21.5,21.2,21.5,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 02:00:00+00:00,nao chove
3,0.0,956.6,957.4,956.6,0.0,21.0,21.0,21.3,20.9,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 03:00:00+00:00,nao chove
4,0.0,957.0,957.0,956.6,0.0,21.3,21.3,21.3,21.0,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 04:00:00+00:00,nao chove


In [42]:
df_test = df.set_index(['Estaçao', 'datahora'])

In [43]:
np.array(list(df_test.groupby('Estaçao').apply(pd.DataFrame.as_matrix)))

AttributeError: type object 'DataFrame' has no attribute 'as_matrix'

In [9]:
from RainPredictionMachine.Trainer import pipe_creator
full_pipe, X_train, y_train = pipe_creator(df)


/Users/isabelagaio/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
X = df.copy()
y = pd.DataFrame(df['classe_chuva'])
#----------------label encoder and to_categorical----------------
label = LabelEncoder()
y_enc = label.fit_transform(y)
y_cat = to_categorical(y_enc)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3) #separando train e test
#----------------scaling pipeline----------------
scaling_pipe = Pipeline([
    ('stdscaler', StandardScaler()),
])
#----------------column transformer----------------
#realizando as operações em paralelo
col_trans = ColumnTransformer([
    ('scaling ', scaling_pipe,[ 'Pres',
                                'Pres_max',
                                'Pres_min',
                                'Radiacao',
                                'Temp',
                                'Temp_orvalho',
                                'Temp_max',
                                'Temp_min',
                                'Temp_orvalho_max',
                                'Temp_orvalho_min',
                                'Umid_max',
                                'Umid_min',
                                'Umid',
                                'Rajada_vento',
                                'Vel_vento',
                                'Dir_vento']), 
    ('pass', 'passthrough', ['Latitude','Longitude','Altitude'])])

full_pipe = Pipeline([
    ('column_stransformer', col_trans),
#    ("deep_learning" , RNN_model ),
])
full_pipe

/Users/isabelagaio/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('column_stransformer',
                 ColumnTransformer(transformers=[('scaling ',
                                                  Pipeline(steps=[('stdscaler',
                                                                   StandardScaler())]),
                                                  ['Pres', 'Pres_max',
                                                   'Pres_min', 'Radiacao',
                                                   'Temp', 'Temp_orvalho',
                                                   'Temp_max', 'Temp_min',
                                                   'Temp_orvalho_max',
                                                   'Temp_orvalho_min',
                                                   'Umid_max', 'Umid_min',
                                                   'Umid', 'Rajada_vento',
                                                   'Vel_vento', 'Dir_vento']),
                                                 ('pass', 'pass

In [24]:
X_train.shape

(6132, 23)

In [11]:
X_train.dtypes

Chuva                           float64
Pres                            float64
Pres_max                        float64
Pres_min                        float64
Radiacao                        float64
Temp                            float64
Temp_orvalho                    float64
Temp_max                        float64
Temp_min                        float64
Temp_orvalho_max                float64
Temp_orvalho_min                float64
Umid_max                        float64
Umid_min                        float64
Umid                            float64
Dir_vento                       float64
Rajada_vento                    float64
Vel_vento                       float64
Estaçao                          object
Latitude                        float64
Longitude                       float64
Altitude                        float64
datahora            datetime64[ns, UTC]
classe_chuva                     object
dtype: object

In [26]:
X_train_t = full_pipe.fit_transform(X_train)

In [27]:
X_train_t.shape

(6132, 19)

In [28]:
# X_train_t1 = np.expand_dims(X_train_t, 0)

In [29]:
# X_train_t1.shape

(1, 6132, 19)

In [30]:
y_train.shape

(6132, 4)

In [ ]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# X_pad = pad_sequences(X_train_t1)
# X_pad = pad_sequences(X_train_t1, dtype= 'float32', padding= 'post', value=-1) 
# X_pad.sort()

In [31]:
model = Sequential()

model.add(LSTM(units=20, activation='tanh', input_shape=(6132, 23)))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accurancy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20)                3520      
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 4)                 44        
                                                                 
Total params: 3,774
Trainable params: 3,774
Non-trainable params: 0
_________________________________________________________________


In [32]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_t1, y_train, batch_size=32, epochs=1000, verbose=1,
         validation_split=0.2,
          callbacks=[es])

ValueError: Training data contains 1 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.